In [1]:
from manim import *
from manim.utils.color import Colors
# from manim_fontawesome import *
from manim_fonts import *
from random import seed, shuffle, normalvariate, choice
import numpy as np
import scipy as sp

%load_ext autoreload
%autoreload 2

from nextgen_01_defs import *
from nextgen_01_lib import *


Manim Community v0.17.2

In [2]:

from manim import *

person = SVGMobject("noun-person-1492700.svg", fill_color=RED)

POS_COLOR = RED
NEG_COLOR = BLUE

SICK_SYMB = "X"
WELL_SYMB = "O"

def patient_state_icon(symb, color=BLACK):
    # with RegisterFont("Open Sans") as fonts:
    #     return Text(symb, font=fonts[0], weight=BOLD, color=color)
    return Text(symb, font="sans-serif", weight=BOLD, color=color)

def title_text(text, color=WHITE):
    with RegisterFont("Montserrat") as fonts:
        return Text(text, font="sans-serif", color=color)

def title(self, text, wait_time=5):
    title = title_text(text)
    self.play(Write(title))
    self.wait(wait_time)
    self.play(Unwrite(title))

def data61_logo(t):
    return SVGMobject('./CSIRO-Data61-logo.svg').scale(1.5)

# 00: Welcome

In [43]:
%%manim -v WARNING --progress_bar None NG_01_00_Welcome

seed(14)

class NG_01_00_Welcome(Scene):
    def construct(self):
        logo = SVGMobject('./CSIRO-Data61-logo.svg').scale(1.5)
        # logo = SVGMobject('./Data61-logo.svg')
        # logo = SVGMobject('./CSIRO-logo.svg')
        self.play(Write(logo))
        self.wait()
        logo.generate_target()
        logo.target.scale(0.30).to_edge(DR)
        self.play(MoveToTarget(
            logo))
        self.wait()
        lesson_text = title_text('NextGen Lecture 01')
        self.play(Write(lesson_text))
        self.wait()
        lesson_text2 = title_text('Beginning probability')
        self.play(Transform(lesson_text, lesson_text2))
        self.wait()
        self.play(FadeOut(lesson_text2))
        dan = ImageMobject('dan_head.webp')
        lesson_text3 = title_text('with Dan MacKinlay').align_to(dan, DOWN)
        self.play(FadeIn(dan))
        self.play(Write(lesson_text3))
        self.wait(2)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 01: Probabilities as proportions



In [24]:
%%manim -v WARNING --progress_bar None NG_01_01_ProbPopulation
SEED = 15

class NG_01_01_ProbPopulation(Scene):

    def construct(self):
        title(self, 'Probabilities and expectations', wait_time=5)

        seed(SEED)
        rng = np.random.default_rng(SEED)
        scale = 0.2
        wait_scale = 1

        def create_person(height = 0.2, color=WHITE):
            return SVGMobject(
                "noun-person-1492700.svg",
                fill_color=color,
                # stroke_width=0.01,
                stroke_color=WHITE,
                height=height,
            ).set_z_index(0)
            return person
    
        pop_group = VGroup(*[
            create_person(
                height = normalvariate(0.2, 0.04),
                color=choice(list(Colors)).value
            )
            for i in range(100)
        ])

        # pop_group = Group(*pop)
        pop_group.arrange_in_grid(n_cols=10)
        pop_group.shift(LEFT*3.25)
        self.play(Create(pop_group))

        q_texts = MathTex(
            r"""P(\text{Studied science}) &=? \\""",
            r"""\mathbb{E}[\text{Income}] &=? \\""",
            r"""\mathbb{E}[\text{Income}\mid\text{Studied science}] &=?\\""",
            r"""P(\text{Changeds the world}\mid\text{Studied science}]) &=?""",
            font_size=45
        ).shift(RIGHT*2.25)
        self.play(Write(q_texts), run_time=5)
        self.wait(5)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 01b: It takes a village

A classic: how to diagnose an illness, given an illness test.

In [56]:
%%manim -v WARNING --progress_bar None NG_01_01b_ProbDoctor
SEED = 15

class NG_01_01b_ProbDoctor(Scene):

    def construct(self):
        title(self, 'The dreaded salmon pox')

        self.n_well_pos = 9
        self.n_well_neg = 81
        self.n_sick_pos = 9
        self.n_sick_neg = 1
        self.n_pop = (
            self.n_well_pos +
            self.n_well_neg +
            self.n_sick_pos +
            self.n_sick_neg
        )
        seed(SEED)
        rng = np.random.default_rng(SEED)

        scale = 0.2
        wait_scale = 1

        well_pos = [patient_state_icon(WELL_SYMB, NEG_COLOR) for i in range(self.n_well_pos)]
        well_neg = [patient_state_icon(WELL_SYMB, NEG_COLOR) for i in range(self.n_well_neg)]
        sick_pos = [patient_state_icon(SICK_SYMB, NEG_COLOR) for i in range(self.n_sick_pos)]
        sick_neg = [patient_state_icon(SICK_SYMB, NEG_COLOR) for i in range(self.n_sick_neg)]
        pop = well_pos + well_neg + sick_pos + sick_neg
        ordered_pop = list(pop)
        shuffle(pop)
        pop_group = Group(*pop)
        pop_group.arrange_in_grid(n_cols=20)
        pop_group.shift(LEFT*3.25)


        census_text = MathTex(
            r"""\text{\# pop.}&=100\\""",
            r"""\text{\# sick}&=10\\""",
            r"""\text{\# well}&=90\\""",
            r"""\text{\# well \& +ve}&=9\\""",
            r"""\text{\# sick \& +ve}&=9\\""",
            r"""\frac{\text{\# sick \& +ve}}{\text{\# +ve}} &= {{ \,?\, }}""",
            # tex_template=TexFontTemplates.gnu_freesans_tx,
            font_size=55)

        (
            pop_text,
            sick_text,
            well_text,
            well_pos_text,
            sick_pos_text,
            true_sick_text,
            true_sick_ans_text
        ) = census_text
        census_text.shift(RIGHT*3.25)
        # self.add(index_labels(census_text))

        well_group = VGroup(*well_pos, *well_neg)
        sick_group = VGroup(*sick_pos, *sick_neg)
        well_pos_group = VGroup(*well_pos)
        well_neg_group = VGroup(*well_neg)
        sick_pos_group = VGroup(*sick_pos)
        sick_neg_group = VGroup(*sick_neg)
        pos_group = VGroup(*well_pos, *sick_pos)
        neg_group = VGroup(*sick_neg, *well_neg)

        self.play(FadeIn(pop_group))
        self.wait(1.0*wait_scale)
        self.play(Write(pop_text))
        self.play(Indicate(pop_group))
        self.play(Write(sick_text))
        self.play(Indicate(sick_group))
        self.wait(1.0*wait_scale)
        self.play(Write(well_text))
        self.play(Indicate(well_group))
        self.wait(1.0*wait_scale)
        self.play(FadeToColor(pos_group, color=POS_COLOR))
        # self.play(Indicate(Group(pos_group, well_pos_text, sick_pos_text), color=POS_COLOR))
        self.wait(1.0*wait_scale)
        self.play(Write(well_pos_text))
        self.play(Indicate(well_pos_group, color=POS_COLOR))
        self.wait(1.0*wait_scale)
        self.play(Write(sick_pos_text))
        self.play(Indicate(sick_pos_group, color=NEG_COLOR))
        self.wait(1.0*wait_scale)
        self.play(Write(VGroup(true_sick_text, true_sick_ans_text)))
        self.wait(1.0*wait_scale)

        whole_pop_group = pop_group.copy()

        self.play(Unwrite(neg_group))
        self.wait(1.0*wait_scale)

        pos_group.generate_target()
        pos_group.target.arrange(LEFT).shift(LEFT*3).scale(0.5)
        self.play(MoveToTarget(
            pos_group))
        self.wait(1.0*wait_scale)
        pos_brace = Brace(pos_group)
        pos_brace_text = MathTex(
            r"""\text{\# +ve}&=18""").next_to(pos_brace, DOWN)
        self.play(Write(
            VGroup(pos_brace, pos_brace_text)
        ))
        self.wait(1.0*wait_scale)
        sick_pos_brace = Brace(sick_pos_group, UP)
        sick_pos_brace_text = MathTex(
            r"""\text{\# sick \& +ve}&=9""").next_to(sick_pos_brace, UP)
        self.play(Write(
            VGroup(sick_pos_brace, sick_pos_brace_text)
        ))
        answer_text = MathTex(
            r"""\frac{9}{18}""",
            font_size=55
        ).move_to(true_sick_ans_text.get_left(), LEFT )
        self.play(Transform(
            true_sick_ans_text,
            answer_text
        ))
        self.wait(1.0*wait_scale)
        better_answer_text = MathTex(
            r"""\frac{1}{2}""",
            font_size=55
        ).move_to(true_sick_ans_text.get_left(), LEFT)
        self.wait(1.0*wait_scale)
        self.play(Transform(
            true_sick_ans_text,
            better_answer_text
        ))
        self.wait(5.0*wait_scale)

        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )

        # census_text_prop = MathTex(
        #     r"""&P(\text{sick}|\text{-ve})\\""",
        #     r"""&=\frac{P(\text{sick \& \text{-ve}})}{P(\text{-ve})}\\""",
        #     r"""&= \frac{1}{82}""",
        #     font_size=55
        # )
        # new_census_text_prop = MathTex(
        #     r"""&P(\text{sick}|\text{-ve})\\""",
        #     r"""&=\frac{P(\text{sick \& \text{-ve}})}{P(\text{-ve})}\\""",
        #     r"""&= \frac{0.01}{0.82}""",
        #     font_size=55
        # )
        # self.play(Write(census_text_prop))
        # self.wait(5)
        # self.play(TransformMatchingTex(census_text_prop, new_census_text_prop))
        # self.wait(5)
        # self.play(Unwrite(new_census_text_prop))
        census_text_prop = MathTex(
            r"""&P(\text{sick}\mid\text{+ve})\\""",
            r"""&=\frac{P(\text{+ve}\mid\text{sick})P(\text{sick})}{P(\text{+ve})}\\""",
            r"""&= \frac{\frac{9}{10}\cdot\frac{10}{100}}{\frac{18}{100}}=\frac{1}{2}""",
            font_size=55
        ).shift(RIGHT)
        self.play(Write(census_text_prop), run_time=5)
        self.wait(5)
        bayes_brace = Brace(census_text_prop, LEFT)
        bayes_brace_text = Text(
            """Bayes’\nTheorem""").next_to(bayes_brace, LEFT)
        self.play(
            Write(bayes_brace), Write(bayes_brace_text)
        )
        
        self.wait(5)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )



# 02: A model of the doctor office

Moving from observations to random generation.

In [55]:
%%manim -v WARNING --progress_bar None NG_01_02_ProbViaSim

SEED = 15

class NG_01_02_ProbViaSim(Scene):
    def construct(self):
        def create_person(marker=WELL_SYMB, color=NEG_COLOR, stroke_color=BLACK):
            return VGroup(
                SVGMobject(
                    "noun-person-1492700.svg",
                    fill_color=color,
                    stroke_color=stroke_color,
                    height=3,
                    width=1.5,
                ).set_z_index(0),
                patient_state_icon(
                    marker, BLACK,
                ).scale(0.75).set_z_index(0.5)
            )
        seed(SEED)
        rng = np.random.default_rng(SEED)
        title(self, 'A doctor’s office')

        DOOR_OFFSET = LEFT * 4
        
        corridor = Rectangle(
            height=4, width=4,
            color=BLACK,
            # stroke_color=WHITE,
            fill_color=BLACK,
            fill_opacity=1.0
        ).shift(DOOR_OFFSET+LEFT*3).set_z_index(1)

        door = Rectangle(height=4, width=2).set_z_index(3).shift(DOOR_OFFSET)

        self.add(corridor)
        self.play(
            FadeIn(door),
        )
        chart = BarChart(
            values=[0,0,0,0],
            bar_names=[WELL_SYMB, WELL_SYMB, SICK_SYMB, SICK_SYMB],
            bar_colors=[NEG_COLOR, POS_COLOR, NEG_COLOR, POS_COLOR],
            y_range=[0, 1.0, 0.1],
            y_length=6,
            x_length=6,
            # x_axis_config={"font_size": 36},
        )
        chart.shift(3*RIGHT)
        self.play(
            FadeIn(chart)
        )
        count = np.array([0, 0, 0, 0])
        for i in range(200):
            # Technically this is not how we should be sampling;
            # we SHOULD be using an urn model.
            # but this will do OK for now
            if i==0:
                p = np.array([0.95, 0.8])
            elif i==1:
                p = np.array([0.94, 0.95])
            elif i==2:
                p = np.array([0.04, 0.95])
            else:
                p = rng.random(2)
            if p[1]<0.9: # well
                symb = WELL_SYMB
                if p[0]>0.9: # pos
                    color = POS_COLOR
                    count[1] += 1
                else:
                    color = NEG_COLOR
                    count[0] += 1
            else:  # sick
                symb = SICK_SYMB
                if p[0]>0.1: # pos
                    color = POS_COLOR
                    count[3] += 1
                else:
                    color = NEG_COLOR
                    count[2] += 1
            if i<30:
                person = create_person(
                    symb, color
                ).shift(DOOR_OFFSET + LEFT*1.5)

                self.add(person)
                self.play(Succession(
                    MoveAlongPath(
                        person,
                        Line(person.get_center(), RIGHT*3.5 + person.get_center())
                    ),
                    chart.animate.change_bar_values(count/count.sum()),
                    FadeOut(person),
                    run_time=3
                ))
            elif i<50:
                person = create_person(
                    symb, color
                ).shift(
                    DOOR_OFFSET + LEFT*1.5 + RIGHT*3.5
                    # + rng.random()*0.25*UP + rng.random()*0.25*RIGHT
                )
                person.set_z_index(-i)
                self.add(person)
                self.play(
                    FadeOut(person),
                    chart.animate.change_bar_values(count/count.sum()),
                    run_time=0.1)
            else:
                person = create_person(
                    symb, color
                ).shift(
                    DOOR_OFFSET + LEFT*1.5 + RIGHT*3.5
                    # + rng.random()*0.25*UP + rng.random()*0.25*RIGHT
                )
                self.add(person)
                self.play(
                    FadeOut(person),
                    chart.animate.change_bar_values(count/count.sum()),
                    run_time=0.05
                )
        self.wait(5)
        
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )

# 03: Probability via simulation

The square dartboard model

In [34]:
%%manim -v WARNING --progress_bar None NG_01_03_ProbViaSim2

SEED = 15

class NG_01_03_ProbViaSim2(Scene):

    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        scale = 0.2
        title(self, "A simulated doctor’s office")
        targetspace = Axes(
            y_range=[0, 1.0, 0.1],
            x_range=[0, 1.0, 0.1],
            y_length=4,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(4*LEFT)
        self.add(targetspace)

        def as_rectangle_corners(bottom_left, top_right):
            return [
                (top_right[0], top_right[1]),
                (bottom_left[0], top_right[1]),
                (bottom_left[0], bottom_left[1]),
                (top_right[0], bottom_left[1]),
            ]
        
        def create_rect(bottom_left, top_right, color=POS_COLOR):
            # Why did I not do this with a Rectangle? Cannot remember
            polygon = Polygon(
                *[
                    targetspace.c2p(*i)
                    for i in as_rectangle_corners(
                        bottom_left, top_right
                    )
                ]
            )
            polygon.stroke_width = 1
            polygon.set_fill(color, opacity=0.3)
            polygon.set_stroke(color)
            return polygon

        def in_zone(zone, point):
            return (
                zone[0][0] <= point[0] < zone[1][0]
                and zone[0][1] <= point[1] < zone[1][1]
            )
        
        icon_scale = 0.6
        well_neg_zone = (0.0, 0.0), (0.9, 0.9)
        well_neg_rect = create_rect(*well_neg_zone, NEG_COLOR)
        self.add(well_neg_rect)
        self.add(patient_state_icon(WELL_SYMB, color=NEG_COLOR).scale(icon_scale).shift(well_neg_rect.get_center()))
        well_pos_zone = (0.9, 0.0), (1.0, 0.9)
        well_pos_rect = create_rect(*well_pos_zone, POS_COLOR)
        self.add(well_pos_rect)
        self.add(patient_state_icon(WELL_SYMB, color=POS_COLOR).scale(icon_scale).shift(well_pos_rect.get_center()))
        sick_neg_zone = (0.0, 0.9), (0.1, 1.0)
        sick_neg_rect = create_rect(*sick_neg_zone, NEG_COLOR)
        self.add(sick_neg_rect)
        self.add(patient_state_icon(SICK_SYMB, color=NEG_COLOR).scale(icon_scale).shift(sick_neg_rect.get_center()))
        sick_pos_zone = (0.1, 0.9), (1.0, 1.0)
        sick_pos_rect = create_rect(*sick_pos_zone, POS_COLOR)
        self.add(sick_pos_rect)
        self.add(patient_state_icon(SICK_SYMB, color=POS_COLOR).scale(icon_scale).shift(sick_pos_rect.get_center()))

        zones = [well_neg_zone, well_pos_zone, sick_neg_zone, sick_pos_zone]
        rects = [well_neg_rect, well_pos_rect, sick_neg_rect, sick_pos_rect]
        count = np.array([0, 0, 0, 0])

        def zone(point):
            for i, zone in enumerate(zones):
                if in_zone(zone, point):
                    return i, zone
            raise ValueError("Point not in any zone")

        chart = BarChart(
            values=[0,0,0,0],
            bar_names=[WELL_SYMB, WELL_SYMB, SICK_SYMB, SICK_SYMB],
            bar_colors=[NEG_COLOR, POS_COLOR, NEG_COLOR, POS_COLOR],
            y_range=[0, 1.0, 0.1],
            y_length=6,
            x_length=6,
            # x_axis_config={"font_size": 36},
        )
        chart.shift(3*RIGHT)
        self.play(
            FadeIn(chart)
        )

        ## various attempts to animate bar labels, all failed
        # c_bar_lbls = chart.get_bar_labels(font_size=24, 
        #     label_constructor=lambda f: DecimalNumber(float(f), num_decimal_places=2))    
        # for i, lab in enumerate(c_bar_lbls):
        #     # lab.add_updater(lambda l: l.set_value(chart.values[i]))
        #     lab.add_updater(lambda l: l.become(chart.bar_labels[i]))
        # self.add(chart, c_bar_lbls)
        # c_bar_lbls.add_updater(lambda l: l.become(chart.bar_labels))
        self.wait(1)
        for j, (n_point_list_now, dur) in enumerate(
                [(1,1.0),] *10 + [(2*k , 1/k) for k in range(1, 25)]):
            new_dots= []
            for step in range(n_point_list_now):
                # override first few point_list so the bars start out even-ish
                if count.sum() == 0:
                    p = np.array([0.45, 0.95])
                elif count.sum() == 1:
                    p = np.array([0.12, 0.8])
                elif count.sum() == 2:
                    p = np.array([0.03, 0.97])
                elif count.sum() == 3:
                    p = np.array([0.93, 0.2])
                elif count.sum() == 4:
                    p = np.array([0.03, 0.97])
                else:
                    p = rng.random(2)
                new_dots.append( Dot(
                    targetspace.c2p(*p),
                    radius=0.08,
                ))
                i, z = zone(p)
                count[i] += 1

            self.add(
                *new_dots
            )
            
            # if count.sum()>13: break
            if n_point_list_now ==1:
                self.play(Indicate(VGroup(*new_dots), scale_factor=2), run_time = dur)
                self.play(
                    chart.animate.change_bar_values(count/count.sum()),
                    # Transform(
                    #     c_bar_lbls,
                    #     chart.bar_labels),
                    Indicate(rects[i],scale_factor=1),
                    run_time=dur)
            else:
                self.play(*[Indicate(dot, scale_factor=2) for dot in new_dots], run_time = dur)
                self.play(
                    chart.animate.change_bar_values(count/count.sum()),
                    run_time=dur)  
            if j in (0,1) :
                self.wait(10)
                
        self.wait(5)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 04: Continuous distributions

In [35]:
%%manim -v WARNING --progress_bar None NG_01_04_ContinuousModels
SEED = 15

class NG_01_04_ContinuousModels(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        title(self, "Continuous Models")
        def create_person(width=1.5, height=2.6, *_, color=PURPLE):
            person = SVGMobject(
                "noun-person-1492700.svg",
                fill_color=color,
                width=width, height=height,
            ).set_z_index(0)
            # Actually the height is ignored so we need to stretch it
            person = person.stretch_to_fit_width(width).stretch_to_fit_height(height)
            return person

        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        
        targetspace.shift(3*RIGHT)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))
        self.play(
            FadeIn(targetspace, labels),
        )
        DOOR_OFFSET = LEFT * 4
        
        corridor = Rectangle(
            height=6, width=4,
            color=BLACK,
            # stroke_color=WHITE,
            fill_color=BLACK,
            fill_opacity=1.0
        ).shift(DOOR_OFFSET+LEFT*3.5).set_z_index(1)

        door = Rectangle(height=6, width=3).set_z_index(3).shift(DOOR_OFFSET)

        self.add(corridor)
        self.play(
            FadeIn(door),
        )

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))
        point_list = []

        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)

            lines = targetspace.get_lines_to_point(targetspace.c2p(width, height))
            # point = Point(targetspace.c2p(width, height))  # draws nothing?
            point = Dot(targetspace.c2p(width, height), radius = 0.05, fill_opacity=0.5)
            point_list.append(point)
            if i==20:
                self.play(
                    FadeOut(
                        door,
                    ),
                    run_time=0.1
                )
            if i < 20:
                person = create_person(
                    width, height
                ).shift(DOOR_OFFSET + LEFT*3)
                self.add(person)
                self.play(
                    MoveAlongPath(
                        person,
                        Line(person.get_center(), targetspace.c2p(width/2, height/2))
                    )
                )

                self.play(
                    Write(
                        lines,
                    ),
                    run_time=0.5
                )
                self.play(Indicate(point), scale_factor=5)
                self.play(
                    FadeOut(person),
                    FadeOut(lines),
                    run_time=0.5
                )
            else:
                self.play(
                    FadeIn(
                        point,
                    ),
                    run_time=0.1
                )
                self.play(Indicate(point), run_time=0.25, scale_factor=5)

        self.wait(5)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )

# 05: Continuous distributions and cumulative probability

In [36]:
%%manim -v WARNING --progress_bar None NG_01_05_ContinuousModelsCDF
SEED = 15

class NG_01_05_ContinuousModelsCDF(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 45
        title(self, "Probabilities in continuous models")
        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))
        t_bottom = 0
        t_left = 0.0
        t_right = 3
        t_top = 4

        t_shirt = SVGMobject(
                "noun-shirt-3972412.svg",
                fill_color=GREEN,
                fill_opacity=0.5,
                width=t_right, height=t_top,
            ).set_z_index(-1).stretch_to_fit_width(t_right).stretch_to_fit_height(t_top)
        t_shirt_box = VGroup(
            t_shirt,
            Rectangle(
                height=t_top, width=t_right,
                stroke_color=GREEN
            )
        ).move_to(targetspace.c2p(t_right/2+t_left, t_top/2+t_bottom))

        def in_t_shirt(obj):
            x, y, *_ = obj.get_center()
            isin =  t_shirt_box.get_left()[0] < x < t_shirt_box.get_right()[0] and t_shirt_box.get_bottom()[1]  < y < t_shirt_box.get_top()[1]
            return isin

        def update_t_shirt(new_top, new_right, new_left=0, new_bottom=0, run_time = 1):
            new_width = new_right - new_left
            new_height = new_top - new_bottom
            t_shirt_box.generate_target()
            t_shirt_box.target.stretch_to_fit_width(new_width).stretch_to_fit_height(new_height)
            t_shirt_box.target.move_to(targetspace.c2p(new_width/2+t_left, new_height/2+t_bottom))
            self.play(MoveToTarget(t_shirt_box), run_time=run_time)

        def color_in_t_shirt(obj):
            if in_t_shirt(obj):
                obj.set_color(GREEN)
            else:
                obj.set_color(WHITE)
        
        self.play(
            FadeIn(targetspace, labels),
        )
        point_list = []
        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)

            lines = targetspace.get_lines_to_point(targetspace.c2p(width, height))
            point = Dot(targetspace.c2p(width, height), fill_opacity=0.5).set_z_index(1)
            color_in_t_shirt(point)
            point.add_updater(color_in_t_shirt)
            point_list.append(point)
        point_group = VGroup(*point_list)
        self.play(FadeIn(point_group, t_shirt_box))
        self.wait()

        t_shirt_prob_text = MathTex(
            # r"P\left( \text{Shirt fits} | {{ \quad }} \right) = \frac{ {{ 0 }} }{100}",
            r"P\left( \text{Shirt fits}| {{ WWW }} \right) = {{ \frac{0 }{100} }}",
            font_size=FONT_SIZE,)
        t_shirt_prob_text.shift(LEFT*3.25 + UP *2)
        p_l, p_cond, p_r, p_frac = t_shirt_prob_text
        mini_t_shirt = t_shirt.copy().match_width(p_cond).stretch_to_fit_height(1.8).move_to(p_cond)
        # p_cond.become(mini_t_shirt)
        p_frac.add_updater(
            lambda m: m.become(
                MathTex(
                    r"\frac{ %d }{100}" % sum(in_t_shirt(p) for p in point_list), font_size=FONT_SIZE
                ).move_to(p_frac)
            )
        )
        self.play(FadeIn(p_l, mini_t_shirt, p_r, p_frac))

        self.wait()

        update_t_shirt(4, 3, run_time=2)

        self.wait()

        update_t_shirt(1, 5, run_time=2)
        self.wait()

        update_t_shirt(5, 1, run_time=2)
        self.wait()

        update_t_shirt(4, 5, run_time=2)
        self.wait()

        update_t_shirt(6, 6, run_time=2)
        self.wait(5)

        # t_shirt_pdf_text = MathTex(
        #     r"P\left( \text{Shirt fits}| {{ WWW }} \right) = {{ \frac{0 }{100} }}",
        #     font_size=FONT_SIZE,)
        # t_shirt_prob_text.shift(LEFT*3.25 + UP *2)
        # p_l, p_cond, p_r, p_frac = t_shirt_prob_text
        # mini_t_shirt = t_shirt.copy().match_width(p_cond).stretch_to_fit_height(1.8).move_to(p_cond)
        # # p_cond.become(mini_t_shirt)
        # p_frac.add_updater(
        #     lambda m: m.become(
        #         MathTex(
        #             r"\frac{ %d }{100}" % sum(in_t_shirt(p) for p in point_list), font_size=FONT_SIZE
        #         ).move_to(p_frac)
        #     )
        # )
        # self.play(FadeIn(p_l, mini_t_shirt, p_r, p_frac))
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 06: Continuous models and expectation

How much do our t-shirts cost?

In [37]:
%%manim -v WARNING --progress_bar None NG_01_06_ContinuousModelsExpectations
SEED = 15

class NG_01_06_ContinuousModelsExpectations(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 35
        TEXT_COLOR = GREEN
        title(self, "expectations")
        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))
        self.play(FadeIn(targetspace, labels))
        
        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))

        point_list = []
        for i, (wh) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = wh
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)
            point = Dot(targetspace.c2p(width, height), fill_opacity=0.5).set_z_index(1)
            point_list.append(point)
        point_group = VGroup(*point_list)

        self.play(FadeIn(point_group))
        self.wait()

        
        exp_text = MathTex(
            r"&{{ \sum }} _{\text{people} } \bigg[ \text{arm span} \times \text{height} \bigg] \\&= {{ \;?\; }}",
            font_size=FONT_SIZE, color=TEXT_COLOR )
        exp_text.shift(LEFT*3.25 + UP *2)
        self.play(FadeIn(exp_text))

        self.wait(5)
        running_sum_text = exp_text[-1]
        nums = ""
        for i, (w,h) in enumerate(vals.T):
            new_nums =  r"{:.2f}\times {:.2f}".format(w,h)
            if i>0:
                new_nums = r" + " + new_nums
            nums = nums + new_nums
            lines = targetspace.get_lines_to_point(targetspace.c2p(w,h))
            self.play( Create(lines))
            point_list[i].set_color(TEXT_COLOR)
            running_sum_text = running_sum_text.become(
                MathTex(
                    nums, font_size=FONT_SIZE, color=TEXT_COLOR
                ).move_to(running_sum_text, LEFT)
            )
            self.play(FadeOut(lines))
            if i> 4: break

        self.wait(2)

        mega_sum = vals[0] @ vals[1]
        point_group.set_color(TEXT_COLOR)
        next_running_sum_text = MathTex(
            "{:.2f}".format(mega_sum),
            font_size=FONT_SIZE, 
            color=TEXT_COLOR
        ).move_to(running_sum_text, LEFT),
        self.play(Transform(running_sum_text, next_running_sum_text[0]))
        self.wait(2)
        next_exp_text = MathTex(
            r"\mathbb{E}",
            font_size=int(FONT_SIZE*1.8), 
            color=TEXT_COLOR
        ).move_to(exp_text[1], LEFT)
        next_running_sum_text = MathTex(
            r"\frac{ " + "{:.2f}".format(mega_sum) + r" }{100}",
            font_size=FONT_SIZE, 
            color=TEXT_COLOR
        ).move_to(running_sum_text, LEFT),
        # print(exp_text[1], next_exp_text, next_running_sum_text)
        self.wait()
        self.play(
            Transform(running_sum_text, next_running_sum_text[0]),
            Transform(exp_text[1], next_exp_text)
        )
        self.play(Indicate(next_exp_text))
        self.wait(2)
        next_running_sum_text = MathTex(
            r"{:.2f}".format(mega_sum/100),
            font_size=FONT_SIZE, 
            color=TEXT_COLOR
        ).move_to(running_sum_text, LEFT),
        self.play(
            Transform(running_sum_text, next_running_sum_text[0]),
        )
        self.play(Indicate(next_exp_text))
        self.wait(2)

        new_exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)} \bigg[\text{arm span}\cdot \text{height} \bigg| \text{height}=3 \bigg] = ?",
            color=BLUE, font_size=FONT_SIZE
        ).move_to(exp_text, UL)

        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )

# 07: Histograms I

In [76]:
%%manim -v WARNING --progress_bar None NG_01_07_ContinuousModelsBinned
SEED = 15

class NG_01_07_ContinuousModelsBinned(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 45
        TEXT_COLOR = BLUE

        title(self, "Binning" )

        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))

        point_list = []

        def create_box_on_axes(x, y, size, color=BLUE):
            w, h, *_ = targetspace.c2p(size, size) - targetspace.c2p(0, 0)
            box = Rectangle(
                width=w,
                height=h,
                color=color,
            ).set_z_index(2)
            text = Text(r" ").set_z_index(3)
            labeled_box = VGroup(
                text,
                box,
            ).set_z_index(5)
            labeled_box.move_to(targetspace.c2p(x, y), DL)
            return labeled_box
        
        def n_points_in_box(box):
            return sum(in_box(p, box) for p in point_list)

        def count_points_in_box(box):
            count = n_points_in_box(box)
            text = box[0]
            text.become(
                count_text_points_in_box(box))

        def count_text_points_in_box(labelbox):
            box = labelbox[1]
            count = n_points_in_box(box)
            if count==0:
                text =  Text(r" ")
                text.z_index = 3
            else:
                text =  MathTex(
                    # r"\mathbf{\frac{%d}{%d}}" % (count, N_POP)
                    r"\mathbf{%d}" % count
                ).scale_to_fit_height(0.6 * box.height)
                text.z_index = 5
            text.move_to(box)
            text.color = box.get_color()
            text.fill_color = box.get_color()
            text.stroke_color = box.get_color()
            labelbox[0] = text
            return text


        def in_box(obj, box):
            x, y, *_ = obj.get_center()
            isin = (
                box.get_left()[0] < x < box.get_right()[0] and
                box.get_bottom()[1]  < y < box.get_top()[1])
            return isin

        self.play(FadeIn(targetspace, labels))

        
        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)

            lines = targetspace.get_lines_to_point(targetspace.c2p(width, height))
            point = Dot(
                targetspace.c2p(width, height),
                fill_opacity=0.5, radius=0.05).set_z_index(1)
            point_list.append(point)
        
        point_group = VGroup(*point_list)

        exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[\text{arm span} \times \text{height} \bigg] \\"
            r"&\approx {{ ? }}",
            color=BLUE, font_size=FONT_SIZE
        ).shift(LEFT*3.25 + UP *2)
        sum_text = exp_text[-1]
        self.play(Write(exp_text))

        for bin_size, wait_scale in [
                (1,1),
                # (0.5,0.25),
                # (0.25,0.25)
                ]:
            sum_text.become(MathTex(
                "0", font_size=FONT_SIZE, color=TEXT_COLOR
            ).move_to(sum_text, LEFT))
            boxes = []
            box_grid = []
            for x in range(int(5/bin_size)):
                box_row = []
                for y in range(int(6/bin_size)):
                    box = create_box_on_axes(x*bin_size, y*bin_size, bin_size)
                    boxes.append(box)
                    box_row.append(box)
                box_grid.append(box_row)
            box_group = VGroup(*boxes)
            
            self.play(Write(point_group))
            self.wait()
            self.play(Write(box_group))
            
            text_list = []
            for box in box_group:
                text_list.append(
                    count_text_points_in_box(box))
            self.play(*[Write(text) for text in text_list])
            
            self.wait()

            megasum = 0.0
            nums = "{:.2f}".format(megasum)
            for box_col in box_grid:
                for box in box_col:
                    count = n_points_in_box(box)
                    self.play(Indicate(box), run_time=0.25*wait_scale)
                    if count > 0:
                        cw, ch, *_ = targetspace.p2c(box.get_center()) 
                        new_nums =  (
                            r"+ {:.2f}\times {:.2f} \times ".format(cw,ch, count) +
                            r"\frac{%d}{%d}" % (count, N_POP)
                        )
                        nums = nums + new_nums
                        megasum = megasum + cw*ch*count/N_POP
                        if wait_scale > 0.25:
                            lines = targetspace.get_lines_to_point(box.get_center())
                            self.play(Succession(Write(lines), Unwrite(lines), run_time=0.5*wait_scale))
                            new_sum_text = MathTex(
                                    nums, font_size=FONT_SIZE, color=TEXT_COLOR
                                ).move_to(sum_text, LEFT)
                            self.play(
                                Transform(sum_text, new_sum_text),
                                run_time=wait_scale
                            )
                            self.wait(wait_scale)
                        nums = r"{:.2f}".format(megasum)
                        new_sum_text = MathTex(
                            nums, font_size=FONT_SIZE, color=TEXT_COLOR
                        ).move_to(sum_text, LEFT)
                        self.play(sum_text.animate.become(new_sum_text), run_time=wait_scale)
            self.wait(3)
            self.play(Unwrite(box_group))
            
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 08: Conditioning in histograms

In [39]:
%%manim -v WARNING --progress_bar None NG_01_08_ContinuousModelsBinnedConditioned
SEED = 15

class NG_01_08_ContinuousModelsBinnedConditioned(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 45
        TEXT_COLOR = BLUE

        title(self, "Conditioning" )

        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))
        new_labels = targetspace.get_axis_labels(x_label=MathTex("w", font_size=35 ), y_label=MathTex("h", font_size=35))
        self.play(FadeIn(targetspace, labels))
        self.wait()
        self.play(labels.animate.become(new_labels))

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))

        point_list = []

        def create_box_on_axes(x, y, size, color=BLUE):
            w, h, *_ = targetspace.c2p(size, size) - targetspace.c2p(0, 0)
            box = Rectangle(
                width=w,
                height=h,
                color=color,
            ).set_z_index(2)
            text = Text(r" ").set_z_index(3)
            labeled_box = VGroup(
                text,
                box,
            ).set_z_index(5)
            labeled_box.move_to(targetspace.c2p(x, y), DL)
            return labeled_box
        
        def n_points_in_box(box):
            return sum(in_box(p, box) for p in point_list)

        def count_points_in_box(box):
            count = n_points_in_box(box)
            text = box[0]
            text.become(
                count_text_points_in_box(box))

        def count_text_points_in_box(labelbox):
            box = labelbox[1]
            count = n_points_in_box(box)
            if count==0:
                text =  Text(r" ")
                text.z_index = 3
            else:
                # text =  MathTex(
                #     # r"\mathbf{\frac{%d}{%d}}" % (count, N_POP)
                #     r"\mathbf{%d}" % count
                # ).scale_to_fit_height(0.7 * box.height)
                text =  Text(
                    # r"\mathbf{\frac{%d}{%d}}" % (count, N_POP)
                    str(count)
                ).scale_to_fit_height(0.7 * box.height)
                text.z_index = 5
            text.move_to(box)
            text.color = box.get_color()
            text.fill_color = box.get_color()
            text.stroke_color = box.get_color()
            labelbox[0] = text
            return text


        def in_box(obj, box):
            x, y, *_ = obj.get_center()
            isin = (
                box.get_left()[0] < x < box.get_right()[0] and
                box.get_bottom()[1]  < y < box.get_top()[1])
            return isin


        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)
            point = Dot(
                targetspace.c2p(width, height),
                fill_opacity=0.5, radius=0.05).set_z_index(1)
            point_list.append(point)
        
        point_group = VGroup(*point_list)

        exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[w \times h \bigg| 3 \leq h < 3.5 \bigg] \\"
            r"&= {{ ? }}",
            color=BLUE, font_size=FONT_SIZE
        ).shift(LEFT*3.25 + UP *2)
        sum_text = exp_text[-1]
        self.play(Write(exp_text))

        bin_size = 0.5
        wait_scale = 1
        boxes = []
        box_grid = []
        for x in range(int(5/bin_size)):
            box_row = []
            for y in range(int(6/bin_size)):
                box = create_box_on_axes(x*bin_size, y*bin_size, bin_size)
                boxes.append(box)
                box_row.append(box)
            box_grid.append(box_row)
        box_group = VGroup(*boxes)
        
        self.play(Create(point_group), Create(box_group))
        
        text_list = []
        for box in box_group:
            text_list.append(
                count_text_points_in_box(box))
        self.play(*[Write(text) for text in text_list])
        
        self.wait()


        box_row_i = 6
        box_row = [box_col[box_row_i] for box_col in box_grid]
        counts = []
        self.play(*[Indicate(box) for box in box_row], run_time=wait_scale)
        for box in box_row:
            count = n_points_in_box(box)
            counts.append(count)

        bin_mids = np.array(list(range(len(box_row))))*bin_size + bin_size/2
        bin_names = [r"{:.2f}".format(mid) for mid in bin_mids]
        cond_chart = BarChart(
            values=counts,
            bar_names=bin_names,
            bar_colors=[BLUE for _ in counts],
            # y_range=[0, 1.0, 0.1],
            y_length=2,
            x_length=6,
            # x_axis_config={"font_size": 36},
            y_axis_config=dict(include_ticks=False, include_numbers=False, numbers_to_include=[]),
        )
        cond_chart.shift(3*LEFT+1.5*DOWN)
        bar_labels = cond_chart.get_bar_labels()
        labeled_chart = VGroup(cond_chart, bar_labels)
        new_box = box_row[4].copy()
        self.add(new_box)
        self.play(
            new_box.animate.become(labeled_chart),
            *[Uncreate(box) for box in boxes if box not in box_row],
        )
        self.wait(5)
        num_sum_string = "+".join([
            # " {{ " + 
            r"{:d}\cdot {:.2f}".format(count, mid) 
            # + " }} " 
            for (count, mid) in zip(counts, bin_mids) if count>0]
        )
        denom_sum_string = "+".join([
            #  " {{ " + 
            r"{:d}".format(count) 
            #  +  " }} " 
            for count in counts if count>0]
        )
        self.play(sum_text.animate.become(MathTex(
            r"\frac{(" + num_sum_string + r")\cdot 3.25}{" + denom_sum_string + r"}",
            font_size=FONT_SIZE*0.6, color=TEXT_COLOR
        ).move_to(sum_text, LEFT)))
        cond_exp = np.array(counts).dot(bin_mids)*3.25/np.sum(counts)
        self.wait(5)
        self.play(sum_text.animate.become(MathTex(r"{:.2f}".format(cond_exp), font_size=FONT_SIZE, color=TEXT_COLOR).move_to(sum_text, LEFT )))

        self.play(Uncreate(box_group))
        
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


KeyError: 'NG_01_07_ContinuousModelsBinnedConditioned'

# 09: Histograms II

In [ ]:
%%manim -v WARNING --progress_bar None NG_01_09_ContinuousModelsBinned
SEED = 15

class NG_01_09_ContinuousModelsBinned(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 45
        TEXT_COLOR = BLUE

        title(self, "Binning" )

        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=Text("arm span", font_size=30 ), y_label=Text("height", font_size=30))

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))

        point_list = []

        def create_box_on_axes(x, y, size, color=BLUE):
            w, h, *_ = targetspace.c2p(size, size) - targetspace.c2p(0, 0)
            box = Rectangle(
                width=w,
                height=h,
                color=color,
            ).set_z_index(2)
            text = Text(r" ").set_z_index(3)
            labeled_box = VGroup(
                text,
                box,
            ).set_z_index(5)
            labeled_box.move_to(targetspace.c2p(x, y), DL)
            return labeled_box
        
        def n_points_in_box(box):
            return sum(in_box(p, box) for p in point_list)

        def count_points_in_box(box):
            count = n_points_in_box(box)
            text = box[0]
            text.become(
                count_text_points_in_box(box))

        def count_text_points_in_box(labelbox):
            box = labelbox[1]
            count = n_points_in_box(box)
            if count==0:
                text =  Text(r" ")
                text.z_index = 3
            else:
                text =  MathTex(
                    # r"\mathbf{\frac{%d}{%d}}" % (count, N_POP)
                    r"\mathbf{%d}" % count
                ).scale_to_fit_height(0.7 * box.height)
                text.z_index = 5
            text.move_to(box)
            text.color = box.get_color()
            text.fill_color = box.get_color()
            text.stroke_color = box.get_color()
            labelbox[0] = text
            return text


        def in_box(obj, box):
            x, y, *_ = obj.get_center()
            isin = (
                box.get_left()[0] < x < box.get_right()[0] and
                box.get_bottom()[1]  < y < box.get_top()[1])
            return isin

        self.play(FadeIn(targetspace, labels))

        
        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)

            lines = targetspace.get_lines_to_point(targetspace.c2p(width, height))
            point = Dot(
                targetspace.c2p(width, height),
                fill_opacity=0.5, radius=0.05).set_z_index(1)
            point_list.append(point)
        
        point_group = VGroup(*point_list)

        exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[\text{arm span} \cdot \text{height} \bigg] \\"
            r"&\approx {{ ? }}",
            color=BLUE, font_size=FONT_SIZE
        ).shift(LEFT*3.25 + UP *2)
        sum_text = exp_text[-1]
        self.play(Write(exp_text))

        for bin_size, wait_scale in [
                # (1,1),
                (0.5,0.25),
                (0.25,0.25)
                ]:
            sum_text.become(MathTex(
                "0", font_size=FONT_SIZE, color=TEXT_COLOR
            ).move_to(sum_text, LEFT))
            boxes = []
            box_grid = []
            for x in range(int(5/bin_size)):
                box_row = []
                for y in range(int(6/bin_size)):
                    box = create_box_on_axes(x*bin_size, y*bin_size, bin_size)
                    boxes.append(box)
                    box_row.append(box)
                box_grid.append(box_row)
            box_group = VGroup(*boxes)
            
            self.play(Write(point_group))
            self.wait()
            self.play(Write(box_group))
            
            text_list = []
            for box in box_group:
                text_list.append(
                    count_text_points_in_box(box))
            self.play(*[Write(text) for text in text_list])
            
            self.wait()

            megasum = 0.0
            nums = "{:.2f}".format(megasum)
            for box_col in box_grid:
                for box in box_col:
                    count = n_points_in_box(box)
                    self.play(Indicate(box), run_time=0.25*wait_scale)
                    if count > 0:
                        cw, ch, *_ = targetspace.p2c(box.get_center()) 
                        new_nums =  (
                            r"+ {:.2f}\times {:.2f} \times ".format(cw,ch, count) +
                            r"\frac{%d}{%d}" % (count, N_POP)
                        )
                        nums = nums + new_nums
                        megasum = megasum + cw*ch*count/N_POP
                        if wait_scale > 0.25:
                            lines = targetspace.get_lines_to_point(box.get_center())
                            self.play(Succession(Write(lines), Unwrite(lines), run_time=0.5*wait_scale))
                            new_sum_text = MathTex(
                                    nums, font_size=FONT_SIZE, color=TEXT_COLOR
                                ).move_to(sum_text, LEFT)
                            self.play(
                                TransformMatchingTex(sum_text, new_sum_text),
                                run_time=wait_scale
                            )
                            self.wait(wait_scale)
                        nums = r"{:.2f}".format(megasum)
                        new_sum_text = MathTex(
                            nums, font_size=FONT_SIZE, color=TEXT_COLOR
                        ).move_to(sum_text, LEFT)
                        self.play(sum_text.animate.become(new_sum_text), run_time=wait_scale)
            self.wait(3)
            self.play(Unwrite(box_group))
            
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


# 10: Densities

In [23]:
%%manim -v WARNING --progress_bar None NG_01_10_ContinuousModelsDensity
SEED = 15

class NG_01_10_ContinuousModelsDensity(Scene):
    def construct(self):
        seed(SEED)
        rng = np.random.default_rng(SEED)

        N_POP = 100
        FONT_SIZE = 35
        TEXT_COLOR = BLUE

        targetspace = Axes(
            y_range=[0, 6, 1.0],
            x_range=[0, 5, 1.0],
            y_length=6,
            x_length=5,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(3*RIGHT).set_z_index(4)
        labels = targetspace.get_axis_labels(x_label=MathTex("w", font_size=30 ), y_label=MathTex("h", font_size=30))
        self.play(FadeIn(targetspace, labels))

        mean = np.array([2, 3.5]).reshape(-1,1)
        covar = np.array([1**2, 1*1.5*0.8, 1*1.5*0.8, 1.5**2]).reshape(2,2) * 0.25
        (covar)
        chol = np.linalg.cholesky(covar)
        vals = mean + chol @ rng.normal(size=(2,100))

        point_list = []
        for i, (vals) in enumerate(vals.T):
            person_shape = np.zeros(3)
            person_shape[:2] = vals
            width, height = person_shape[:2]
            # # should be the same
            axis_person_shape = targetspace.c2p(width, height) - targetspace.c2p(0, 0)
            point = Dot(targetspace.c2p(width, height), fill_opacity=0.5).set_z_index(1)
            point_list.append(point)

        point_group = VGroup(*point_list)
        self.play(FadeIn(point_group))
        self.wait()

        exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[\text{arm span} \cdot \text{height} \bigg] \\"
            r"&= {{ ? }}",
            color=BLUE, font_size=FONT_SIZE
        ).shift(LEFT*3.25 + UP *2)

        self.play(Write(exp_text))
        self.wait(3)
        new_exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[w\cdot h \bigg] \\"
            r"&= {{ ? }}",
            color=BLUE, font_size=FONT_SIZE
        ).move_to(exp_text, UL)
        self.play(exp_text.animate.become(new_exp_text))
        
        
        ## This nightmare is how we get a density field in manim:
        rv = sp.stats.multivariate_normal(mean=mean.ravel(), cov=covar)
        pos = np.dstack(np.meshgrid(np.linspace(0, 5, 100), np.linspace(0, 6, 100)))
        density = rv.pdf(pos)
        density /= density.max()
        density *= 255 * 1.25
        density = np.flip(density, 0)
        density = np.minimum(density, 255)
        density = density.astype(np.uint8) 
        # # Was hoping this would add an alpha channel, doesn't seem to though
        # density = np.repeat(np.expand_dims(density, -1), 4, axis=2)
        densityim = ImageMobject(density ).shift(3*RIGHT)
        densityim.set_resampling_algorithm(RESAMPLING_ALGORITHMS["cubic"])
        densityim.stretch_to_fit_height(6).stretch_to_fit_width(5).set_z_index(1).align_to(targetspace.c2p(0,0), DL)
        # print(densityim.get_height(), densityim.get_width())
        # print(densityim.get_left()[0], densityim.get_right()[0])
        # print(densityim.get_bottom()[1], densityim.get_top()[1])
        self.play(FadeIn(densityim))
        self.play(FadeOut(point_group))
        self.wait()

        ## Isaac Newton never appears. Not sure why.
        newton = SVGMobject("noun-isaac-newton-365274.svg").scale(2)
        print(newton)
        self.play(Create(newton))
        self.wait(3)
        self.play(FadeOut(newton))

        int_text = exp_text[-1]

        new_int_text = MathTex(
            r""" \iint 
            %\limits_{\text{span} \times \text{height}}
            {{ w }} \cdot {{ h }} \cdot {{ p(w, h) }} \, \mathrm{d}w \, \mathrm{d}h""",
            font_size=FONT_SIZE, color=TEXT_COLOR
        ).move_to(int_text, LEFT )

        self.play(
            Transform(int_text, new_int_text)
        )
        self.wait(3)

        self.play(Uncreate(exp_text), Uncreate(new_int_text))

        new_exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[w\cdot h \bigg| h=3 \bigg] \\"
            r"""&= \frac{\int  w  \cdot  3  \cdot  p(w, 3)  \, \mathrm{d}h}{\int \cdot  p(w, 3)  \, \mathrm{d}w """,
            color=BLUE, font_size=FONT_SIZE
        ).move_to(exp_text, UL)

        self.play(
            Write(new_exp_text)
        )
        self.wait(5)

        def transect(h, grid_size=0.5):
            ws = np.arange(0, 5+0.01, grid_size)
            h0s = np.ones_like(ws) * h
            h1s = np.ones_like(ws) * (h + grid_size)
            intd = rv.cdf(
                np.stack([ws[1:], h1s[1:]], axis=1),
                lower_limit=np.stack([ws[:-1], h0s[:-1]], axis=1),
            )
            denom = intd.sum()
            return intd/denom
        
        def rect_plot(vals, grid_size=0.5):
            rect_list = []
            for i, val in enumerate(vals):
                height = val/grid_size
                rect = Rectangle(
                    height=height,
                    width=grid_size,
                    color=GREEN,
                    fill_opacity=0.5,
                    fill_color=GREEN).shift(i*grid_size*RIGHT + height/2*UP)
                rect.z_index = 5
                rect_list.append(rect)
            line = Line(ORIGIN, grid_size*len(vals)*RIGHT, color=GREEN).shift(grid_size*len(vals)*RIGHT/2)
            rect_plot = VGroup(line, *rect_list).shift(LEFT*0.5*len(vals)*grid_size)
            return rect_plot

        rect0 = Rectangle(
            height=1, width=5, color=GREEN, fill_opacity=0.5, fill_color=GREEN,grid_xstep=1.0, grid_ystep=1.0).shift(targetspace.c2p(5/2, 3+1/2))
        rect0.z_index = 5

        transect0 = rect_plot(transect(3, grid_size=1.0)).shift(3*LEFT+1*DOWN)
        self.add(transect0)

        self.play(Write(rect0))
        self.wait(1)
        for grid_size in [0.5, 0.25, 0.125, 0.0625]:
            next_rect = Rectangle(
                height=grid_size, width=5, color=GREEN, fill_opacity=0.5, fill_color=GREEN,grid_xstep=grid_size, grid_ystep=grid_size).shift(targetspace.c2p(5/2, 3+grid_size/2))
            next_rect.z_index = 5
            next_transect = rect_plot(
                transect(3, grid_size=grid_size),
                grid_size).shift(3*LEFT+1*DOWN)
            self.wait(1)
            self.play(
                Transform(rect0, next_rect),
                Transform(transect0, next_transect),
                run_time=1)

        next_rect = Rectangle(
            height=0, width=5, color=GREEN, fill_opacity=0.5, fill_color=GREEN,grid_xstep=grid_size, grid_ystep=grid_size).shift(targetspace.c2p(5/2, 3))
        self.play(Transform(rect0, next_rect), run_time=1)

        self.wait(5)
        self.play(Unwrite(rect0), Unwrite(transect0))

        self.play(Unwrite(new_exp_text))
        self.wait(3)

        new_exp_text = MathTex(
            r"&\mbox{\Large\( \mathbb{E}\)}_{\text{people} }\bigg[\phi(w,h) \bigg| h=h^* \bigg] \\"
            r"""&= \frac{\int \phi( w ,h^*)  p(w, h^*)  \, \mathrm{d}h}{\int \cdot  p(w, h^*)  \, \mathrm{d}w """,
            color=BLUE, font_size=FONT_SIZE
        ).move_to(exp_text, UL)
        self.play(
            Write(new_exp_text)
        )
        self.wait(10)
        self.play(Unwrite(new_exp_text))

        gaussian_text = MathTex(
            r"""&p\left(\begin{bmatrix}w\\h\end{bmatrix}\right)\\
            &=\frac{e^{-\frac{1}{2}\left(\begin{bmatrix}w\\h\end{bmatrix}-\begin{bmatrix}m_w\\m_h\end{bmatrix}\right)^{\top} \boldsymbol{\Sigma}^{-1}\left(\begin{bmatrix}w\\h\end{bmatrix}-\begin{bmatrix}m_w\\m_h\end{bmatrix}\right)}}{\sqrt{\operatorname{det}(2 \pi \boldsymbol{\Sigma})}}
            """,
            color=BLUE, font_size=FONT_SIZE*0.8
        ).shift(LEFT*3)
        self.play(
            Write(gaussian_text)
        )
        self.wait(10)
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )


SVGMobject
